# Data Pre-processing:

Converting the default *Military Aircraft Detection Dataset* in to yolo format dataset

In [ ]:
%pip install glob2 pytest-shutil cv-3 os.path2 tqdm

  Using cached glob2-0.7.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached pytest_shutil-1.7.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached cv-3-1.2.0.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 13.5 MB/s eta 0:00:0000:0100:01
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9301 sha256=b1042a30a491d68ae8d1f0b5300df0694f18edda9e1830dfc9ba26b354601c07
  Stored in directory: /Users/taneya-pavitra/Library/Caches/pip/wheels/96/41/06/9f8fddc6eb1d75bde63db7f491311a4ae26905212617e06eb2
  Created wheel for cv-3: filename=cv_3-1.2.0-py3-none-any.whl size=12704 sha256=3a9c34cf3156e94881e10b626ff3b3a324b56c7a870bb6309610d310b0987efc
  Stored in directory: /Users/taneya-pavitra/Library/Caches/pip/wheels/3c/84/4e/970d343aa1552eb7cb51bc435fba69690c0baa6dfb506fc17f
  Created wheel for os.path2: filename=os.path2-0.0.4-py3-none-any.whl size=7

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
import cv2
import os
import yaml
from tqdm import tqdm

In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Setup complete ✅ (20 CPUs, 31.8 GB RAM, 1131.9/1862.4 GB disk)


In [ ]:
# Different type of Aircrafts
classes = np.array(['A10','A400M','AG600','AV8B',
        'B1','B2','B52','Be200',
        'C130','C2','C17','C5','E2','E7','EF2000',
        'F117','F14','F15','F16','F18','F22','F35','F4',
        'JAS39','MQ9','Mig31','Mirage2000','P3','RQ4','Rafale',
        'SR71','Su34','Su57',
        'Tu160','Tu95','Tornado',
        'U2','US2', 'V22','XB70','YF23','Vulcan','J20',
        'KC135', 'J10', 'Su25', 'Su24']
)

# Number of classes
print(len(classes))

47


**Using the files from dataset folder**

In [ ]:
# Paths and directories to be used in next step for conversion of original csv's into yolo format txt files
csv_paths = glob.glob('dataset/*.csv')
jpg_paths = glob.glob('dataset/*.jpg')
csv_paths.sort()
jpg_paths.sort()

# Number of images and Number of jpgs files
print(f'num of images: {len(csv_paths)} \nnum of jpgs: {len(jpg_paths)}')

# Creating folders for each class
for cls in classes:
  os.makedirs(f'yolo_classes/images/{cls}', exist_ok=True)
  os.makedirs(f'yolo_classes/labels/{cls}', exist_ok=True)

num of images: 13442 
num of jpgs: 13442


**Default csv labels: width, height, class, xmin, ymin, xmax, ymax**

**Target label structure: class_num, x_center, y_center, width, heigth**

In [ ]:
# Converting original dataset csv files into yolo format dataset txt files
for i, (csv_path, jpg_path) in enumerate(tqdm(zip(csv_paths, jpg_paths))):
    annotations = np.array(pd.read_csv(csv_path))
    # print(f'Annotations: {annotations}')

    cls = ''
    for annot in annotations:
      cls = class_name = annot[3]

    jpg_file_path = f'yolo_classes/images/{cls}/' + os.path.basename(jpg_path)
    txt_file_path = f'yolo_classes/labels/{cls}/' + os.path.basename(csv_path)[:-4]+'.txt'
    shutil.copy(jpg_path, jpg_file_path)
    # print(f'Images and Text files - {jpg_file_path}, {txt_file_path}')

    with open(txt_file_path, mode='w') as f:
        try:
            for annotation in annotations:
                width = annotation[1]
                height = annotation[2]
                class_name = annotation[3]

                # Calculating the x/y values for x_center and y_center
                xmin = annotation[4]
                ymin = annotation[5]
                xmax = annotation[6]
                ymax = annotation[7]
                x_center = 0.5*(xmin+xmax)
                y_center = 0.5*(ymin+ymax)

                b_width = xmax - xmin
                b_height= ymax - ymin
                class_num = np.where(classes==class_name)[0][0]
                output_string = '{} {} {} {} {}\n'.format(class_num,
                                                        x_center/width,
                                                        y_center/height,
                                                        b_width/width,
                                                        b_height/height)
                # print(f'Output String: {output_string}')
                f.write(output_string)
        except:
            print(f'text file path: {txt_file_path}')
            0/0

print('finished')

In [ ]:
# Comparing lengths of yolo_classes images/txts with original dataset ones
print('yolo_classes jpg',len(glob.glob('yolo_classes/images/*/*.jpg')))
print('yolo_classes labels',len(glob.glob('yolo_classes/labels/*/*.txt')))
print('Original dataset jpg',len(glob.glob('dataset/*.jpg')))
print('Original dataset labels',len(glob.glob('dataset/*.csv')))

yolo_classes jpg 13442
yolo_classes labels 13442
Original dataset jpg 13442
Original dataset labels 13442


In [ ]:
!pip install split-folders

In [ ]:
# Split the dataset with a ratio 80, 10, 10 for modes: train, val, test resp.
import splitfolders

input_folder = 'yolo_classes/images'
output_folder = 'yolo_format_dataset/images'
input_folder_labels = 'yolo_classes/labels'
output_folder_labels = 'yolo_format_dataset/labels'

# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .1, .1)`.
splitfolders.ratio(input_folder, output=output_folder, seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False) # default values
splitfolders.ratio(input_folder_labels, output=output_folder_labels, seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False) # default values

Copying files: 13442 files [01:06, 203.39 files/s]
Copying files: 13442 files [00:09, 1357.25 files/s]


In [ ]:
# Checking lengths of yolo_format_dataset images
print('yolo_format_dataset train',len(glob.glob('yolo_format_dataset/images/train/*/*.jpg')))
print('yolo_format_dataset val',len(glob.glob('yolo_format_dataset/images/val/*/*.jpg')))
print('yolo_format_dataset test',len(glob.glob('yolo_format_dataset/images/test/*/*.jpg')))
print('Images count', len(glob.glob('yolo_format_dataset/images/train/*/*.jpg')) + len(glob.glob('yolo_format_dataset/images/val/*/*.jpg')) + len(glob.glob('yolo_format_dataset/images/test/*/*.jpg')))

# Checking lengths of yolo_format_dataset labels
print('yolo_format_dataset train',len(glob.glob('yolo_format_dataset/labels/train/*/*.txt')))
print('yolo_format_dataset val',len(glob.glob('yolo_format_dataset/labels/val/*/*.txt')))
print('yolo_format_dataset test',len(glob.glob('yolo_format_dataset/labels/test/*/*.txt')))
print('Label counts', len(glob.glob('yolo_format_dataset/labels/train/*/*.txt')) + len(glob.glob('yolo_format_dataset/labels/val/*/*.txt')) + len(glob.glob('yolo_format_dataset/labels/test/*/*.txt')))

yolo_format_dataset train 10735
yolo_format_dataset val 1320
yolo_format_dataset test 1387
Images count 13442
yolo_format_dataset train 10735
yolo_format_dataset val 1320
yolo_format_dataset test 1387
Label counts 13442


In [ ]:
# Verifying whether the split dataset is correctly divided for both jpg's and txt files for corresponding mode's (train/val/test)
MODE=['train', 'val', 'test']
counter = 0
for mode in MODE:
    txt_paths = glob.glob(f'yolo_format_dataset/labels/{mode}/*/*.txt')
    jpgs = glob.glob(f'yolo_format_dataset/images/{mode}/*/*.jpg')
    txt_paths.sort()
    jpgs.sort()
    for i, (txt_path, jpg) in enumerate(tqdm(zip(txt_paths, jpgs))):
        jpg_file_path = os.path.basename(jpg)[:-4]
        txt_file_path = os.path.basename(txt_path)[:-4]
        counter += 1
        # print(f'{jpg_file_path} and {txt_file_path}')
        if jpg_file_path != txt_file_path:
            print(f'{txt_file_path} is a mismatch')
        # elif jpg_file_path == txt_file_path:
        #     print(f'{txt_file_path} is in {txt_path} and {jpg}')

print(counter)

10735it [00:00, 513905.76it/s]
1320it [00:00, 465446.09it/s]
1387it [00:00, 500947.18it/s]

13442


Rearranging the final input dataset for training will be in below format:



    yolo_final_dataset/

        images/
            train/*.jpg
            val/*.jpg
            test/*.jpg

        labels/
            train/*.txt
            val/*.txt
            test/*.txt  

In [ ]:
# Create a final dataset which has to be in the yolo format -> yolo_final_dataset/images/{mode:train/val/test}/*.jpg
for mode in MODE:
    os.makedirs(f'yolo_final_dataset/images/{mode}', exist_ok=True)
    os.makedirs(f'yolo_final_dataset/labels/{mode}', exist_ok=True)
    img_path_src = glob.glob(f'yolo_format_dataset/images/{mode}/*/*.jpg')
    txt_path_src = glob.glob(f'yolo_format_dataset/labels/{mode}/*/*.txt')
    img_path_dest = glob.glob(f'yolo_final_dataset/images/{mode}/.')
    txt_path_dest = glob.glob(f'yolo_final_dataset/labels/{mode}/.')
    # print(img_path_dest[0])
    # print(txt_path_dest[0])
    for img in img_path_src:
        # print(img)
        shutil.copy(img, img_path_dest[0])
    for txt in txt_path_src:
        # print(txt)
        shutil.copy(txt, txt_path_dest[0])

In [ ]:
# Checking lengths of yolo_final_dataset images
print('yolo_final_dataset train',len(glob.glob('yolo_final_dataset/images/train/*.jpg')))
print('yolo_final_dataset val',len(glob.glob('yolo_final_dataset/images/val/*.jpg')))
print('yolo_final_dataset test',len(glob.glob('yolo_final_dataset/images/test/*.jpg')))
print('Images count', len(glob.glob('yolo_final_dataset/images/train/*.jpg')) + len(glob.glob('yolo_final_dataset/images/val/*.jpg')) + len(glob.glob('yolo_final_dataset/images/test/*.jpg')))

# Checking lengths of yolo_final_dataset labels
print('yolo_final_dataset train',len(glob.glob('yolo_final_dataset/labels/train/*.txt')))
print('yolo_final_dataset val',len(glob.glob('yolo_final_dataset/labels/val/*.txt')))
print('yolo_final_dataset test',len(glob.glob('yolo_final_dataset/labels/test/*.txt')))
print('Label counts', len(glob.glob('yolo_final_dataset/labels/train/*.txt')) + len(glob.glob('yolo_final_dataset/labels/val/*.txt')) + len(glob.glob('yolo_final_dataset/labels/test/*.txt')))

yolo_final_dataset train 10735
yolo_final_dataset val 1320
yolo_final_dataset test 1387
Images count 13442
yolo_final_dataset train 10735
yolo_final_dataset val 1320
yolo_final_dataset test 1387
Label counts 13442


In [ ]:
# Deleting folder recursively even for read-only files
def rmtree(path):
    if not os.path.exists(path):
        return 'Folder does not exist'
    DIR_READ_WRITE = 0o700
    FILE_READ_WRITE = 0o600
    def remove_readonly(func, path, _):
        "Clear the readonly bit and reattempt the removal"
        os.chmod(path, FILE_READ_WRITE)
        func(path)

    # set all directories as read/write
    os.chmod(path, DIR_READ_WRITE)
    for root, dirs, files in os.walk(path):
        for d in dirs:
            os.chmod(os.path.join(root, d), DIR_READ_WRITE)

    shutil.rmtree(path, onerror=remove_readonly)

In [ ]:
# No need for yolo_classes and yolo_format_dataset folders anymore and therefore deleting them
rmtree('yolo_classes')
rmtree('yolo_format_dataset')

'Folder does not exits'

In [ ]:
# Creating yaml file
def generate_yaml(train_path, val_path, names, nc, base):
    data = {
        "train": train_path,
        "val": val_path,
        "names": names,
        "nc": nc
    }

    with open(pt.join(base, 'training_data.yaml'), 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)

img_path_train = '/notebooks/AML24/yolo_final_dataset/images/train'
img_path_val = '/notebooks/AML24/yolo_final_dataset/images/val'
base = '/notebooks/AML24/'
Alabels = classes.tolist()
names = {i: name for i, name in enumerate(Alabels)}
nc = len(Alabels)
generate_yaml(img_path_train, img_path_val, names, nc, base)

In [ ]:
# zip the yolo_final_dataset folder
shutil.make_archive('yolo_final_dataset', 'zip', 'yolo_final_dataset')

'/Users/taneya-pavitra/Documents/Personal/Learning-Study/MSc in Artificial Intelligence - University of Surrey/Year Feb 24 - Feb 25/Semester 2/Applied Machine Learning (EEEM068)/CourseWork/Aircraft_Classification/yolo_final_dataset.zip'

In [ ]:
# No need for yolo_final_dataset folder as well and therefore deleting it
rmtree('yolo_final_dataset')

'Folder does not exits'

# Training:

Using the yolo_final_dataset from 'data pre-processing step' as an input for training

In [3]:
from ultralytics import YOLO

## Model: yolov8m.pt
Epochs: 50

Other hyper-parameters are being kept as default

In [4]:
yolo8_m_model = YOLO("yolov8m.pt")
yolo5_m_model = YOLO("yolov5mu.pt")

### Without Augmentation:

    hsv_h: 0
    hsv_s: 0
    hsv_v: 0
    degrees: 0
    translate: 0
    scale: 0
    shear: 0.0
    perspective: 0.0
    flipud: 0.0
    fliplr: 0
    bgr: 0.0
    mosaic: 0
    mixup: 0.0
    copy_paste: 0.0
    auto_augment: null
    erasing: 0
    crop_fraction: 1.0

In [8]:
results = yolo8_m_model.train(data='datasetfile.yaml', epochs=50, device = 0, hsv_h=0, hsv_s=0, hsv_v=0, degrees=0, translate=0, scale=0, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0, bgr=0.0, mosaic=0, mixup=0.0, copy_paste=0.0, auto_augment=None, erasing=0,)

Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=datasetfile.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\train.cache... 10729 images, 0 backgrounds, 6 corrupt: 100%|██████████| 10735/10735 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\2790a93ba1210aaaa09931b951d2376a.jpg: ignoring corrupt image/label: image file is truncated (2 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\4075b1896e913e85d31001f6fbe2336e.jpg: ignoring corrupt image/label: image file is truncated (27 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7e722a1478e888c2d0b8bffeb7bbb154.jpg: ignoring corrupt image/label: image file is truncated (1 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7f35d5ac8df852e80c8a35c061ec9473.jpg: ignoring corrupt image/label: image file is truncated (0 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\e704da53011ed94a1644bec7ffc904e9.jpg: ignoring corrupt image/labe


val: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\val.cache... 1320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1320/1320 [00:00<?, ?it/s]


Plotting labels to runs\detect\train12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000196, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      7.16G    0.07055    0.07777     0.7672         26        640: 100%|██████████| 671/671 [03:23<00:00,  3.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.53it/s]

                   all       1320       2239      0.726      0.508      0.604      0.546



50 epochs completed in 3.052 hours.
Optimizer stripped from runs\detect\train12\weights\last.pt, 52.1MB
Optimizer stripped from runs\detect\train12\weights\best.pt, 52.1MB

Validating runs\detect\train12\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 218 layers, 25866973 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.79it/s]


                   all       1320       2239       0.73      0.507      0.604      0.545
                   A10       1320         59      0.822      0.661       0.76      0.705
                 A400M       1320         43      0.718      0.488      0.635      0.594
                 AG600       1320         20      0.883      0.755      0.904      0.853
                  AV8B       1320         40      0.809       0.53      0.643      0.582
                    B1       1320         60       0.76      0.686      0.717      0.635
                    B2       1320         57       0.81      0.737      0.806      0.724
                   B52       1320         47      0.597      0.596      0.626      0.587
                 Be200       1320         24      0.903        0.5      0.663      0.593
                  C130       1320        101      0.693      0.564      0.641      0.565
                    C2       1320         81      0.859       0.84      0.885      0.847
                   C1

### Augmentation: Flip Left/Right

    fliplr: 0.5

### Augmentation: Resize
    scale: 0.5

In [10]:
results = yolo8_m_model.train(data='datasetfile.yaml', epochs=50, device = 0, hsv_h=0, hsv_s=0, hsv_v=0, degrees=0, translate=0, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0, bgr=0.0, mosaic=0, mixup=0.0, copy_paste=0.0, auto_augment=None, erasing=0,)

Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=datasetfile.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\train.cache... 10729 images, 0 backgrounds, 6 corrupt: 100%|██████████| 10735/10735 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\2790a93ba1210aaaa09931b951d2376a.jpg: ignoring corrupt image/label: image file is truncated (2 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\4075b1896e913e85d31001f6fbe2336e.jpg: ignoring corrupt image/label: image file is truncated (27 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7e722a1478e888c2d0b8bffeb7bbb154.jpg: ignoring corrupt image/label: image file is truncated (1 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7f35d5ac8df852e80c8a35c061ec9473.jpg: ignoring corrupt image/label: image file is truncated (0 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\e704da53011ed94a1644bec7ffc904e9.jpg: ignoring corrupt image/labe


val: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\val.cache... 1320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1320/1320 [00:00<?, ?it/s]


Plotting labels to runs\detect\train13\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000196, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train13
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      7.15G     0.2413     0.2039     0.8097         26        640: 100%|██████████| 671/671 [03:24<00:00,  3.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:12<00:00,  3.45it/s]

                   all       1320       2239       0.83      0.629      0.743      0.676



50 epochs completed in 3.091 hours.
Optimizer stripped from runs\detect\train13\weights\last.pt, 52.1MB
Optimizer stripped from runs\detect\train13\weights\best.pt, 52.1MB

Validating runs\detect\train13\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 218 layers, 25866973 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:10<00:00,  4.09it/s]


                   all       1320       2239      0.832      0.628      0.743      0.676
                   A10       1320         59       0.91      0.853      0.883      0.782
                 A400M       1320         43      0.887      0.605      0.809      0.754
                 AG600       1320         20      0.947       0.75      0.952      0.906
                  AV8B       1320         40      0.823       0.58      0.794      0.705
                    B1       1320         60      0.759      0.682      0.737      0.661
                    B2       1320         57      0.854      0.825       0.89      0.811
                   B52       1320         47      0.818      0.702      0.754      0.663
                 Be200       1320         24      0.937      0.619      0.781      0.717
                  C130       1320        101      0.808      0.614      0.757      0.661
                    C2       1320         81      0.916      0.936      0.959       0.93
                   C1

### Combined/Default Augmentation:

    hsv_h: 0.015
    hsv_s: 0.7
    hsv_v: 0.4
    degrees: 0.0
    translate: 0.1
    scale: 0.5
    shear: 0.0
    perspective: 0.0
    flipud: 0.0
    fliplr: 0.5
    bgr: 0.0
    mosaic: 1.0
    mixup: 0.0
    copy_paste: 0.0
    auto_augment: randaugment
    erasing: 0.4
    crop_fraction: 1.0

In [6]:
results = yolo8_m_model.train(data='datasetfile.yaml', epochs=150, device = 0)

Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=datasetfile.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\train.cache... 10729 images, 0 backgrounds, 6 corrupt: 100%|██████████| 10735/10735 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\2790a93ba1210aaaa09931b951d2376a.jpg: ignoring corrupt image/label: image file is truncated (2 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\4075b1896e913e85d31001f6fbe2336e.jpg: ignoring corrupt image/label: image file is truncated (27 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7e722a1478e888c2d0b8bffeb7bbb154.jpg: ignoring corrupt image/label: image file is truncated (1 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7f35d5ac8df852e80c8a35c061ec9473.jpg: ignoring corrupt image/label: image file is truncated (0 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\e704da53011ed94a1644bec7ffc904e9.jpg: ignoring corrupt image/labe


val: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\val.cache... 1320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1320/1320 [00:00<?, ?it/s]


Plotting labels to runs\detect\train14\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train14
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    150/150      7.11G     0.2428     0.1688     0.8065         14        640: 100%|██████████| 671/671 [03:19<00:00,  3.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:13<00:00,  3.02it/s]

                   all       1320       2239      0.906       0.79      0.877      0.805



150 epochs completed in 9.227 hours.
Optimizer stripped from runs\detect\train14\weights\last.pt, 52.1MB
Optimizer stripped from runs\detect\train14\weights\best.pt, 52.1MB

Validating runs\detect\train14\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 218 layers, 25866973 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.65it/s]


                   all       1320       2239      0.907       0.79      0.877      0.805
                   A10       1320         59      0.877      0.843      0.949       0.85
                 A400M       1320         43          1      0.668      0.884      0.802
                 AG600       1320         20      0.937          1       0.99      0.969
                  AV8B       1320         40      0.971      0.845      0.932      0.876
                    B1       1320         60       0.92       0.85      0.894      0.792
                    B2       1320         57      0.946      0.919      0.945      0.854
                   B52       1320         47       0.78      0.753        0.8      0.713
                 Be200       1320         24      0.895       0.75      0.854      0.821
                  C130       1320        101      0.893      0.741      0.846      0.747
                    C2       1320         81      0.948      0.926      0.971      0.944
                   C1

### YOLOv5 Default Augmentation:

In [6]:
results = yolo5_m_model.train(data='datasetfile.yaml', epochs=50)

Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov5mu.pt, data=datasetfile.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\train... 10729 images, 0 backgrounds, 6 corrupt: 100%|██████████| 10735/10735 [00:08<00:00, 1237.77it/s]

train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\2790a93ba1210aaaa09931b951d2376a.jpg: ignoring corrupt image/label: image file is truncated (2 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\4075b1896e913e85d31001f6fbe2336e.jpg: ignoring corrupt image/label: image file is truncated (27 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7e722a1478e888c2d0b8bffeb7bbb154.jpg: ignoring corrupt image/label: image file is truncated (1 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\7f35d5ac8df852e80c8a35c061ec9473.jpg: ignoring corrupt image/label: image file is truncated (0 bytes not processed)
train: WARNING ⚠️ C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\images\train\e704da53011ed94a1644bec7ffc904e9.jpg: ignoring corrupt image/labe

train: New cache created: C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\train.cache


val: Scanning C:\Users\James\Desktop\Files\Education\Surrey\AML\datasetfinal\labels\val... 1320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1320/1320 [00:01<00:00, 666.89it/s]


...

      50/50      6.78G     0.3244     0.2597     0.8425         26        640: 100%|██████████| 671/671 [03:25<00:00,  3.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.67it/s]

                   all       1320       2239      0.883      0.739      0.847      0.769



50 epochs completed in 2.976 hours.
Optimizer stripped from runs\detect\train11\weights\last.pt, 50.5MB
Optimizer stripped from runs\detect\train11\weights\best.pt, 50.5MB

Validating runs\detect\train11\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv5m summary (fused): 248 layers, 25072429 parameters, 0 gradients, 64.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:12<00:00,  3.37it/s]


                   all       1320       2239      0.883       0.74      0.847      0.769
                   A10       1320         59      0.856       0.78      0.906       0.81
                 A400M       1320         43          1      0.675      0.889        0.8
                 AG600       1320         20      0.907       0.98      0.986      0.961
                  AV8B       1320         40      0.938       0.85      0.898      0.838
                    B1       1320         60      0.841      0.733      0.856      0.729
                    B2       1320         57      0.872      0.842      0.908      0.817
                   B52       1320         47      0.837      0.702      0.795      0.706
                 Be200       1320         24          1      0.664      0.819      0.755
                  C130       1320        101      0.866      0.693      0.839      0.741
                    C2       1320         81      0.936      0.926      0.962      0.926
                   C1